In [1]:
!pip install text_hammer
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.18 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.9.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import text_hammer as th
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,TFBertModel
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import shutil
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

MessageError: ignored

In [ ]:
# Загружаем наш датасет
df_train = pd.read_csv('/content/drive/MyDrive/train_text.csv')
df_val = pd.read_csv('/content/drive/MyDrive/validation_text.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test_text.csv')

In [ ]:
# Убираем suprise
df_train = df_train.loc[df_train['label'] != 5]
df_val = df_val.loc[df_val['label'] != 5]
df_test = df_test.loc[df_test['label'] != 5]

In [ ]:
df_full = pd.concat([df_train,df_test,df_val], axis = 0)
df_full.head(10)

In [ ]:
# Смотрим что удаление suprise прошло успешно
df_full['label'].unique()

# 2. Очищаем текст от шума


In [ ]:
# df_cleaned = text_preprocessing(df_full,'text')
# Очищаем наши текста с помощью следующих команд
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df_full['text'] = df_full['text'].progress_apply(lambda x:str(x).lower())
df_full['text'] = df_full['text'].progress_apply(lambda x: th.cont_exp(x)) 
df_full['text'] = df_full['text'].progress_apply(lambda x: th.remove_emails(x))
df_full['text'] = df_full['text'].progress_apply(lambda x: th.remove_html_tags(x))
df_full['text'] = df_full['text'].progress_apply(lambda x: th.remove_special_chars(x))
df_full['text'] = df_full['text'].progress_apply(lambda x: th.remove_accented_chars(x))
df_full['text'] = df_full['text'].progress_apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
df_full

In [ ]:
df_full = df_full.copy()

In [ ]:
df_full['number_words'] = df_full['text'].apply(lambda x:len(x.split()))

In [ ]:
# changing the data type to the category to encode into codes 
df_full['label'] = df_full['label'].astype('category')

In [ ]:
df_full['label'].cat.codes

In [ ]:
encoded_dict  = {'sad':0,'joy':1, 'love':2, 'anger':3, 'fear':4}

In [ ]:
df_full.number_words.max()

In [ ]:
df_full

In [ ]:
data_train, data_test = train_test_split(df_full, test_size=0.2, random_state=42, stratify=df_full['label'])

# Разделение обучающего набора на новый обучающий набор и валидационный набор
data_train, data_val = train_test_split(data_train, test_size=0.2, random_state=42, stratify=data_train['label'])

In [ ]:
data_train.shape

In [ ]:
data_val.shape

In [ ]:
data_test.shape

In [ ]:
to_categorical(data_train['label'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

In [ ]:
tokenizer('Привет')

In [ ]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=data_train['text'].tolist(),
    add_special_tokens=True,
    max_length=40,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

x_val = tokenizer(
    text=data_val['text'].tolist(),
    add_special_tokens=True,
    max_length=40,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=data_test['text'].tolist(),
    add_special_tokens=True,
    max_length=40,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [ ]:
x_test['input_ids']

In [ ]:
max_len = 40
 
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

embeddings = bert(input_ids,token_type_ids, attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(5,activation = 'sigmoid')(out)
    
model1 = tf.keras.Model(inputs=[input_ids, token_type_ids, input_mask], outputs=y)
model1.layers[2].trainable = True

In [ ]:
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-07
    )

loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),

model1.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
model1.summary()

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

In [ ]:
training = model1.fit(
    x ={'input_ids':x_train['input_ids'], 'token_type_ids':x_train['token_type_ids'], 'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train['label']),
    validation_data = (
    {'input_ids':x_val['input_ids'], 'token_type_ids':x_val['token_type_ids'], 'attention_mask':x_val['attention_mask']}, to_categorical(data_test['label'])
    ),
  epochs=1,
    batch_size=512
)

In [ ]:
predicted_raw = model1.predict({'input_ids':x_test['input_ids'], 'token_type_ids':x_test['token_type_ids'], 'attention_mask':x_test['attention_mask']})
y_predicted = np.argmax(predicted_raw, axis = 1)
score = accuracy_score(data_test['label'],y_predicted)
score

121/121 [==============================] - 101s 830ms/step


0.9375162043038631

In [ ]:
print(classification_report(data_test['label'], y_predicted))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1160
           1       0.99      0.92      0.95      1352
           2       0.81      0.97      0.88       328
           3       0.94      0.90      0.92       542
           4       0.94      0.94      0.94       475

    accuracy                           0.94      3857
   macro avg       0.92      0.94      0.93      3857
weighted avg       0.94      0.94      0.94      3857



# 5. Prediction on custom text

In [ ]:
texts = input()

x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=40,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True) 
validation = model1.predict({'input_ids':x_val['input_ids'], 'token_type_ids':x_val['token_type_ids'], 'attention_mask':x_val['attention_mask']})*100
validation

I began to play football and have fun
1/1 [==============================] - 0s 352ms/step


array([[49.180973, 79.51203 , 33.78484 , 55.74509 , 48.0159  ]],
      dtype=float32)

In [ ]:
{'input_ids':x_val['input_ids']}

{'input_ids': <tf.Tensor: shape=(1, 40), dtype=int32, numpy=
 array([[ 101,  146, 1567, 1128,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0]], dtype=int32)>}

In [ ]:
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

sad 16.78166
joy 63.705456
love 93.92863
anger 32.679565
fear 23.303757


In [ ]:
model

In [ ]:
model.save("/content/drive/MyDrive/model_text.h5")